# Data Science in Practice 2020

*MGT-415*

Ref : [https://www.kaggle.com/cgueret/covid-19-risk-factor-predictor/output](https://www.kaggle.com/cgueret/covid-19-risk-factor-predictor/output)

## Project

### Descriptive report

Authors :
- Rayan Chaouche
- Yann Martinson
- Christopher Padovani
- Jules Triomphe

### 1. Initialization

#### Load modules

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns

import random

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier

from ipyparallel import Client
from ipyparallel.joblib import IPythonParallelBackend
from joblib import Parallel, parallel_backend, register_parallel_backend

#### Prepare parallel kernel

*Install [here](https://ipyparallel.readthedocs.io/en/latest/), define the number of engines and click '**Start**' in the* **iPython Clusters** *tab.*

In [2]:
c = Client(profile='default')
print('profile:', c.profile)
print("IDs:", c.ids) # Process id numbers
bview = c.load_balanced_view()
register_parallel_backend('ipyparallel',
                          lambda : IPythonParallelBackend(view=bview))

profile: default
IDs: [0, 1, 2, 3]


---

**THE FOLLOWING CODE IS ADAPTED FROM [https://www.kaggle.com/cgueret/covid-19-risk-factor-predictor](https://www.kaggle.com/cgueret/covid-19-risk-factor-predictor)**

#### Load disease list

- `efo_id` is the disease code from the *Experimental Factor Ontology (EFO)* and *MONDO* ontologies.
- `disease_full_name` is the full name of the disease.
- `number_of_associations` is the number of [associations](https://docs.targetvalidation.org/getting-started/getting-started/associated-targets).

In [3]:
disease_list = pd.read_csv('https://storage.googleapis.com/open-targets-data-releases/20.02/output/20.02_disease_list.csv.gz',
                           compression='gzip')
disease_list.head()

,efo_id,disease_full_name,number_of_associations
0,EFO_1000984,inflammatory breast carcinoma,174
1,MONDO_0004093,esophageal basaloid carcinoma,1
2,EFO_0006352,laryngeal squamous cell carcinoma,796
3,EFO_1000514,Salivary Gland Adenosquamous Carcinoma,0
4,EFO_1001965,pharyngeal squamous cell carcinoma,428


Let's check that there are no duplicate diseases.

In [4]:
disease_list.disease_full_name.value_counts()

negative regulation of circadian sleep/wake cycle, REM sleep                           1
respiratory aspiration                                                                 1
Corneoiridogoniodysgenesis                                                             1
Timothy syndrome                                                                       1
Steatocystoma multiplex - natal teeth                                                  1
                                                                                      ..
anti-citrullinated protein antibody seropositivity                                     1
malignant renal pelvis neoplasm                                                        1
Pfeiffer syndrome type 1                                                               1
Autosomal dominant disease with diffuse palmoplantar keratoderma as a major feature    1
porphyrin metabolism disease                                                           1
Name: disease_full_na

#### Load Kaggle data

- `subject` is the paper id.
- `predicate` is the relation to the disease.
- `object` is the disease id.

In [5]:
raw_kaggle_data = pd.read_csv('COVID_KG_sample.csv')
raw_kaggle_data.describe()

,subject,predicate,object
count,327834,327834,327834
unique,20831,6,9577
top,ENSG00000141510,isAssociatedTo,EFO_0000618
freq,1041,176088,7918


In [6]:
print(raw_kaggle_data.columns)

print(raw_kaggle_data.predicate.value_counts())

Index(['subject', 'predicate', 'object'], dtype='object')
isAssociatedTo              176088
hasGeneticClue              103103
belongsToTherapeuticArea     22142
isASpecific                  17548
isAboutDisease                5986
isAboutTarget                 2967
Name: predicate, dtype: int64


#### Create the training, testing and validation sets

We will validate our model by using the `hasGeneticClue` predicate. Therefore, all other predicates are moved to the training set.

In [7]:
train_set = raw_kaggle_data[raw_kaggle_data['predicate']!='hasGeneticClue'].values
print(type(train_set))
genetic_clue_triples = raw_kaggle_data[raw_kaggle_data['predicate']=='hasGeneticClue']
print(type(genetic_clue_triples))

<class 'numpy.ndarray'>
<class 'pandas.core.frame.DataFrame'>


By definition, we have the following definitions for each predicate :
- `isAboutGene` connects a CORDI-19 paper to a target gene.
- `isAboutDisease` connects a CORDI-19 paper to a Disease (i.e. a characteristic, condition, or behaviour - following OpenTargets terminology).
- `isASsociatedTo` connects a target gene to a Disease (i.e. a characteristic, condition, or behaviour - following OpenTargets terminology).
- `belongsToTherapeuticArea` associates a Disease to its therapeutic area, as provided by Open Targets.
- `isASpecific` this predicate associates a Disease to an higher-up element in the Open Targets hierarchy (ontological path connecting higher classes of diseases to more specific instances).
- `hasGeneticClue` this predicate connects a Disease to a characteristic, condition, or behaviour if there are genetic evidences that such connection exists.

Hence, only `isAboutDisease` and `isASsociatedTo` are associated to a disease. Hence, we can build the used disease list from these two predicates.

train_set_diseases =  np.unique(np.concatenate([
    np.unique(train_set[train_set[:, 1] == 'isAboutDisease'][:, 2]),
    np.unique(train_set[train_set[:, 1] == 'isAssociatedTo'][:, 2]),
], 0))
print('There are %s diseases in the training set.' %(len(train_set_diseases)))

Let's take 5% of these diseases to validate our model.

test_set_diseases = np.random.choice(list(train_set_diseases), 255).tolist()
test_set = genetic_clue_triples[genetic_clue_triples["subject"].isin(test_set_diseases)]

Let's remove these from the training set and randomize the training set.

train_genetic_clue_triples = genetic_clue_triples[~genetic_clue_triples["subject"].isin(test_set_diseases)]
train_set = np.concatenate([train_set, train_genetic_clue_triples], 0)
train_set = np.random.permutation(train_set)
train_set_diseases =  np.unique(np.concatenate([
    np.unique(train_set[train_set[:, 1] == 'isAboutDisease'][:, 2]),
    np.unique(train_set[train_set[:, 1] == 'isAssociatedTo'][:, 2]),
    np.unique(train_set[train_set[:, 1]=='hasGeneticClue'][:, 0]),
    np.unique(train_set[train_set[:, 1]=='hasGeneticClue'][:, 2]),
], 0))

print('There are %s elements in the training set.' %(len(train_set)))
print(train_set.view())
print(type(train_set))
print(test_set_diseases)
print(type(test_set_diseases))
print('There are %s elements in the testing set.' %(len(test_set)))
print('There are %s elements in the raw data set.' %(len(raw_kaggle_data)))

**UP TO THIS POINT, THE CODE WAS ADAPTED FROM [https://www.kaggle.com/cgueret/covid-19-risk-factor-predictor](https://www.kaggle.com/cgueret/covid-19-risk-factor-predictor)**

**WHAT COMES NEXT IS OUR OWN WORK PRODUCT**

---

In [8]:
genetic_clue_triples.describe()

,subject,predicate,object
count,103103,103103,103103
unique,2231,1,5160
top,EFO_0000558,hasGeneticClue,Orphanet_70589
freq,687,103103,552


In [9]:
def df_pre_process():
    
    df_raw = genetic_clue_triples.copy()
    
    # output into dummies

    dummy_dict = { key : ('1' if value == 'MONDO_0100096' else '0')
              for key, value
              in zip(df_raw['object'].values, df_raw['object'].values)}
    df_raw.object.replace(dummy_dict, inplace = True)
    df_raw.rename(columns = {'object' : 'covid'}, inplace = True)
    
    # X y splitting
    
    y = df_raw.covid.copy()
    X_raw = df_raw.drop(columns = ['covid','predicate']).copy()
    
    # input into dummies
    
    X_raw_types = dict(X_raw.dtypes)
    features = list(X_raw.columns)
    categorical_features = [feat for feat in features if X_raw_types[feat] == 'O']
    
    X = pd.get_dummies(X_raw, columns = categorical_features,prefix_sep=':')
    
    return df_raw, categorical_features, X, y

In [10]:
def df_preprocessing_knn() :

    df_raw, categorical_features, X, y = df_pre_process()
    
    # train val splitting

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20)

    y_train = y_train.tolist()
    y_test = y_test.tolist()

    return df_raw, categorical_features, X, X_train, X_test, y, y_train, y_test

In [11]:
def df_preprocessing_rf(test_size) :
    
    df_raw, categorical_features, X, y = df_pre_process()
    
    # train val splitting

    train = np.random.rand(len(df_raw))> test_size

    X_train = X[train]
    X_test = X[~train]

    y_train = y[train].tolist()
    y_test = y[~train].tolist()

    return df_raw, categorical_features, X, X_train, X_test, y, y_train, y_test

In [12]:
def plot_importance(feature_importance_sorted, n, type_of_search):
    
    plt.figure(figsize=(15,5))
    x = np.arange(n)
    y = [feature_importance_sorted[i][1] for i in range(n)]
    labels = [feature_importance_sorted[i][0] for i in range(n)]
    ax = sns.barplot(y,x,orient="h");
    plt.xlabel("Importance fraction", fontsize = 12)
    ax.set_xticklabels(['{:,.0%}'.format(x) for x in ax.get_xticks()])
    plt.yticks(x,labels, fontsize = 15)
    plt.title('Most important feature: {}'.format(type_of_search), fontsize = 15)
    plt.show()

In [13]:
df_raw, categorical_features, X, X_train, X_test, y, y_train, y_test = df_preprocessing_knn()

We run simulations for k-values between 1 and 100 to find the best fit parameters.

In [ ]:
error_uni = np.array([0])
error_dist = np.array([0])

for i in range(1, 100):
    knn_uni = KNeighborsClassifier(n_neighbors=i, weights='uniform', n_jobs=-1)
    knn_uni.fit(X_train, y_train)
    pred_i_uni = knn_uni.predict(X_test)
    error_uni = np.append(error_uni, np.array(np.mean(pred_i_uni != y_test)))

    knn_dist = KNeighborsClassifier(n_neighbors=i, weights='distance', n_jobs=-1)
    knn_dist.fit(X_train, y_train)
    pred_i_dist = knn_dist.predict(X_test)
    error_dist = np.append(error_dist, np.array(np.mean(pred_i_dist != y_test)))


plt.figure(figsize=(12, 6))
plt.plot(error_uni[1:], color='red', linestyle='dashed', marker='o', markerfacecolor='blue', markersize=10, label='Uniform weights')
plt.plot(error_dist[1:], color='black', linestyle='dashed', marker='o', markerfacecolor='green', markersize=10, label='Distance weights')
plt.title('Error Rate K Value')
plt.xlabel('K Value')
plt.ylabel('Mean Error')
plt.legend(loc='best')
plt.show()

print('Minimum error rate with uniform weights: {:.2%} for k = {}'.format(min(error_uni[1:]), np.argmin(error_uni[1:]) + 1))
print('Minimum error rate with distance weights: {:.2%} for k = {}'.format(min(error_dist[1:]), np.argmin(error_dist[1:]) + 1))

In [ ]:
classifier_uni = KNeighborsClassifier(n_neighbors = (np.argmin(error_uni[1:]) + 1), weights='uniform')
classifier_uni.fit(X_train, y_train)
classifier_dist = KNeighborsClassifier(n_neighbors = (np.argmin(error_dist[1:]) + 1), weights='distance')
classifier_dist.fit(X_train, y_train)

y_pred_uni = classifier_uni.predict(X_test)
y_pred_dist = classifier_dist.predict(X_test)

In [ ]:
plt.figure(figsize=(8,8))
sns.heatmap(confusion_matrix(y_test, y_pred_uni), annot=True, fmt="d", cmap="nipy_spectral")
plt.ylabel('Actual label');
plt.xlabel('Predicted label');
plt.title('Confusion matrix for the uniform weights KNN method (k = {})'.format(np.argmin(error_uni[1:]) + 1), fontsize = 15);
print("UNIFORM WEIGHTS")
print("Accuracy: {:.2%}".format(accuracy_score(y_test, y_pred_uni)))
print(classification_report(y_test, y_pred_uni))

In [ ]:
plt.figure(figsize=(8,8))
sns.heatmap(confusion_matrix(y_test, y_pred_dist), annot=True, fmt="d", cmap="nipy_spectral")
plt.ylabel('Actual label');
plt.xlabel('Predicted label');
plt.title('Confusion matrix for the distance weights KNN method (k = {})'.format(np.argmin(error_dist[1:]) + 1), fontsize = 15);
print("DISTANCE WEIGHTS")
print("Accuracy: {:.2%}".format(accuracy_score(y_test, y_pred_dist)))
print(classification_report(y_test, y_pred_dist))

### 3. Random Forest Classifier

We will compare two methods, which are grid search and random search.

In [14]:
df_raw, categorical_features, X, X_train, X_test, y, y_train, y_test = df_preprocessing_rf(0.3)

In [15]:
clf = RandomForestClassifier(n_jobs=-1)

#### 3.1 Grid search

To avoid having too high a computational time, we will focus on 2 of the mot important parameters that are max depth and the number of estimators.

##### Max Depth

This parameter is the depth of the trees, which is one of the most important. We range it between 4 (anything lower seems too low and increases computational time without much results) and 15.

##### Number of estimators

This parameter is the number of trees that are going to be generated. Here the choice of the number of trees will mostly affect the computational time. Let's set the values between 10 and 500 and see the effects.

In [16]:
max_depth = list(range(4,16))

In [17]:
n_estimators = [10, 15, 20, 50, 100, 200, 500]

Let's use the default 5 folds of cross validation.

In [18]:
grid_parameters = {'max_depth' : max_depth, 'n_estimators' : n_estimators }

In [19]:
grid_clf = GridSearchCV(clf, param_grid = grid_parameters, return_train_score=True, verbose = 3)

In [20]:
print(X_train.shape)
print(len(y_train))

(72222, 2231)
72222


In [ ]:
with parallel_backend('ipyparallel'):
    grid_clf.fit(X_train, y_train);

Fitting 5 folds for each of 84 candidates, totalling 420 fits


[Parallel(n_jobs=-1)]: Using backend IPythonParallelBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  24 tasks      | elapsed:  1.1min
[Parallel(n_jobs=-1)]: Done 120 tasks      | elapsed:  5.3min
[Parallel(n_jobs=-1)]: Done 280 tasks      | elapsed: 11.8min


Let's check which model is the best.

In [ ]:
grid_best_score = grid_clf.best_score_
grid_best_parameters = grid_clf.best_params_
grid_best_max_depth = grid_best_parameters.get('max_depth')
grid_best_n_estimators = grid_best_parameters.get('n_estimators')

print('Grid search best_score: {:.5}'.format(grid_best_score))
print('best_max_depth: {}'.format(grid_best_max_depth))
print('best_n_estimators: {}'.format(grid_best_n_estimators))

In [ ]:
grid_clf_best = RandomForestClassifier(n_jobs = -1,max_depth = grid_best_max_depth, n_estimators = grid_best_n_estimators )

In [ ]:
grid_clf_best.fit(X_train, y_train);

Let's apply it to our validation set.

In [ ]:
grid_y_pred = grid_clf_best.predict(X_test)

In [ ]:
print("Accuracy: {:.2%}".format(accuracy_score(y_test, grid_y_pred)))

Given this accuracy, we can take a deeper look into the results.

In [ ]:
grid_cm = confusion_matrix(y_test, grid_y_pred)
index = ['Negative','Positive']  
columns = ['Negative','Positive']  
cm_df = pd.DataFrame(grid_cm,columns,index) 


plt.figure(figsize=(8,8))
sns.heatmap(cm_df, annot=True, fmt="d", cmap="nipy_spectral")
plt.ylabel('Actual label');
plt.xlabel('Predicted label');
plt.title('Confusion matrix for the grid search', fontsize = 15);

##### Feature importance

In [ ]:
grid_feature_importances = [(list(X.columns)[i], grid_clf_best.feature_importances_[i]) for i in range(len(list(X.columns)))]
grid_feature_importances.sort(key=itemgetter(1), reverse = True)
plot_importance(grid_feature_importances, 10, 'Grid search')

In [ ]:
print(classification_report(y_test,grid_y_pred))

##### Feature selection

Let's try to run the model again, but this time selecting only the most impacting features to save us some work and let's compare the results.

In [ ]:
grid_selected_features = [grid_feature_importances[i][0] for i in range(15)]
grid_X_train_sel = X_train[grid_selected_features]
grid_X_test_sel = X_test[grid_selected_features]

In [ ]:
with parallel_backend('ipyparallel'):
    grid_clf.fit(grid_X_train_sel, y_train);

In [ ]:
grid_best_score_sel = grid_clf.best_score_
grid_best_parameters_sel = grid_clf.best_params_
grid_best_max_depth_sel = grid_best_parameters.get('max_depth')
grid_best_n_estimators_sel = grid_best_parameters.get('n_estimators')
print('Grid search best_score with selected features: {:.5}'.format(grid_best_score_sel))
print('Grid search best_max_depth with selected features: {}'.format(grid_best_max_depth_sel))
print('Grid search best_n_estimators with selected features: {}'.format(grid_best_n_estimators_sel))

In [ ]:
grid_clf_best_sel = RandomForestClassifier(n_jobs = -1,max_depth = grid_best_max_depth_sel, n_estimators = grid_best_n_estimators_sel )

In [ ]:
grid_clf_best_sel.fit(grid_X_train_sel, y_train);

In [ ]:
grid_y_pred_sel = grid_clf_best_sel.predict(grid_X_test_sel)

In [ ]:
print("Accuracy: {:.2%}".format(accuracy_score(y_test, grid_y_pred_sel)))

#### 3.2 Random Search

After having explored a grid search, we can adopt another approach. Instead of searching for each value, let's give our model more parameters input, but instead let it choose randomly at each iteration one value for each parameter. It will then be evaluated again.

In [ ]:
n_estimators = range(10,1000,50)
criterion = ['gini', 'entropy']
max_features = ['sqrt', 'log2', None]
bootstrap = [True,False]
max_depth = range(5,50,10)
min_samples_leaf = range(2,100, 2)
min_samples_split = range(2,100,2)

random_parameters = {'n_estimators':n_estimators, 'max_features':max_features, 'max_depth':max_depth, 'min_samples_leaf':
              min_samples_leaf}


In [ ]:
random_clf = RandomizedSearchCV(clf, param_distributions = random_parameters, n_iter = 20, verbose = 3)

In [ ]:
with parallel_backend('ipyparallel'):
    random_clf.fit(X_train, y_train);

In [ ]:
random_best_score = random_clf.best_score_
print('Random search best_score: {:.4}'.format(random_best_score))
random_best_parameters = random_clf.best_params_

In [ ]:
random_best_parameters

In [ ]:
random_clf_best = RandomForestClassifier(max_depth = 35, max_features = 'sqrt', min_samples_leaf = 9, n_estimators = 250)

In [ ]:
random_clf_best.fit(X_train, y_train);

In [ ]:
random_y_pred = random_clf_best.predict(X_test)

In [ ]:
random_cm = confusion_matrix(y_test, random_y_pred)
annot_kws = {"ha": 'center',"va": 'center'}


plt.figure(figsize=(8,8))
sns.heatmap(cm_df, annot=True, fmt="d", cmap="nipy_spectral")
plt.ylabel('Actual label');
plt.xlabel('Predicted label');

In [ ]:
random_feature_importances = [(list(X.columns)[i], random_clf_best.feature_importances_[i]) for i in range(len(list(X.columns)))]
random_feature_importances.sort(key=itemgetter(1), reverse = True)
plot_importance(random_feature_importances, 10, 'Ransom search')

In [ ]:
print(classification_report(y_test,random_y_pred))

In [ ]:
print("Accuracy: {:.2%}".format(accuracy_score(y_test, random_y_pred)))

#### 3.3 Comparison

##### ROC Curve

In [ ]:
fpr_grid, tpr_grid, _ = roc_curve(y_test, grid_y_pred)
fpr_grid_sel, tpr_grid_sel, _ = roc_curve(y_test, grid_y_pred_sel)
fpr_random, tpr_random, _ = roc_curve(y_test, random_y_pred)

In [ ]:
plt.figure(figsize=(12, 6))
plt.plot([0, 1], [0, 1], 'k--')
plt.plot(fpr_grid, tpr_grid, label='Grid Search')
plt.plot(fpr_grid_sel, tpr_grid_sel, label='Grid Search with Selected Predictors')
plt.plot(fpr_random, tpr_random, label='Random Search')
plt.xlabel('False positive rate')
plt.ylabel('True positive rate')
plt.title('ROC curve')
plt.legend(loc='best')
plt.show()

roc_auc_grid = roc_auc_score(y_test, grid_y_pred)
print('ROC AUC for Grid Search: %.5f' % roc_auc_grid)
roc_auc_grid_sel = roc_auc_score(y_test, grid_y_pred_sel)
print('ROC AUC for Grid Search with selected predictors: %.5f' % roc_auc_grid_sel)
roc_auc_random = roc_auc_score(y_test, random_y_pred)
print('ROC AUC for Random Search: %.5f' % roc_auc_random)

We observe that all three methods are better than a random prediction. The Grid Search with Selected Predictors has slightly better prediction than the Random Search, which has in turn slightly better prediction than the basic Grid Search.

In [ ]:
plt.figure(figsize=(12, 6))
plt.xlim(0.05, 0.15)
plt.ylim(0.4, 0.55)
plt.plot([0, 1], [0, 1], 'k--')
plt.plot(fpr_grid, tpr_grid, label='Grid Search')
plt.plot(fpr_grid_sel, tpr_grid_sel, label='Grid Search with Selected Predictors')
plt.plot(fpr_random, tpr_random, label='Random Search')
plt.xlabel('False positive rate')
plt.ylabel('True positive rate')
plt.title('ROC curve')
plt.legend(loc='best')
plt.show()

##### Precision - Recall Curve

In [ ]:
grid_precision, grid_recall, _ = precision_recall_curve(y_test, grid_y_pred)
grid_precision_sel, grid_recall_sel, _ = precision_recall_curve(y_test, grid_y_pred_sel)
random_precision, random_recall, _ = precision_recall_curve(y_test, random_y_pred)

In [ ]:
y_test = np.array(y_test)
no_skill = len(y_test[y_test == 1]) / len(y_test)

plt.figure(figsize=(12, 6))
plt.plot([0, 1], [no_skill, no_skill], linestyle='--', label='No Skill')
plt.plot(grid_recall, grid_precision, marker='.', label='Grid Search')
plt.plot(grid_recall_sel, grid_precision_sel, marker='.', label='Grid Search')
plt.plot(random_recall, random_precision, marker='.', label='Random Search')
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.legend(loc='best')
plt.show()

pr_auc_grid = auc(grid_recall, grid_precision)
print('Precision-Recall AUC for Grid Search: %.5f' % pr_auc_grid)
pr_auc_grid_sel = auc(grid_recall_sel, grid_precision_sel)
print('Precision-Recall AUC for Grid Search with selected predictors: %.5f' % pr_auc_grid_sel)
pr_auc_random = auc(random_recall, random_precision)
print('Precision-Recall AUC for Random Search: %.5f' % pr_auc_random)